12/03...温度なし、エラーが取れた

12/10...温度をつけた、結果の表示方法変更

In [1]:
#import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random, math, warnings
from sklearn.feature_selection import SelectKBest, f_regression

In [2]:
#valiable
warnings.resetwarnings()

warnings.simplefilter('error')
num_select=1
#T=0.1
MCMC_step=10000
T=[j for j in reversed(range(1, MCMC_step+1))]

In [3]:
X = pd.read_table('training_markers.txt', header=None,  sep ='\t')# 説明変数（こっちの特徴量上位5つ取り出す）


data_yields = pd.read_table('training_yields.txt', header=None,  sep ='\t')# 目的変数
y_list = []
for i in range(len(data_yields)):
    y_list.append(data_yields.iloc[i, 0])
y = pd.Series(y_list)


num_sample = X.shape[0]#サンプル数
num_feature = X.shape[1]#特徴量の数

In [4]:
type(X) #pandas.core.frame.DataFrame
X.shape #(20, 20)

type(y) #pandas.core.series.Series
y.shape #(20,)

X

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0,0,1,1,1,1,1,1,0,1,1,0,1,1,0,0,1,0,1,0
1,1,1,1,1,1,1,1,1,0,0,1,0,1,0,0,1,1,0,0,1
2,1,1,0,1,0,1,0,1,0,0,0,1,0,1,0,1,0,1,0,0
3,1,1,0,0,0,0,0,0,0,1,1,0,0,1,0,1,0,0,1,0
4,0,0,1,0,1,0,1,0,1,0,0,1,1,1,1,0,1,1,0,1
5,0,0,1,0,1,1,0,0,1,0,0,0,1,1,1,0,1,0,0,1
6,0,1,0,0,0,1,0,1,0,1,1,1,0,1,1,0,1,1,1,0
7,1,1,0,1,0,0,1,1,0,1,1,0,0,0,0,1,0,1,1,1
8,1,0,0,0,1,1,0,1,1,0,0,0,1,0,0,1,0,0,0,0
9,0,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,1


In [5]:
#特徴量の分散や同じ特徴量が存在するのかを確認

from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=0)
sel.fit(X)

# get_supportで保持するデータのみをTrue値、そうでないものはFalse値を返します
print(sum(sel.get_support())) #20、すなわち同じ特徴量は存在していないことがわかった

20


In [6]:
type(y)

pandas.core.series.Series

In [7]:
def selection(num_select, X, y):
    # num_select個の特徴量を選択
    #print(f_regression(X,y))
    #print(np.linalg.norm(y))
    selector = SelectKBest(score_func=f_regression, k=num_select) #SelectKBest
    selector.fit(X, y)
    mask = selector.get_support()    # 各特徴量を選択したか否かのmaskを取得
    return mask

In [8]:
"""
0-division error occurs when all values in y are 0 or 1
reason: y = y - np.mean(y), corr /= np.linalg.norm(y)
"""
mask = selection(num_select, X, y)
print(mask)

[False False False False False False False False False False False False
 False False False False False False False  True]


In [9]:
"""
def each_energy(mask, y):
    y_list = y.values.tolist()
    energy_find_dic = {}#bestな特徴量:それを選んだときのエネルギー基準値（０となる）
    for i in range(len(mask)):
        if mask[i]==True:
            energy_find_trait = np.dot(np.array(X[:][i]).T, np.array(y_list))
            energy_find_dic[i]=energy_find_trait
    return sum(energy_find_dic.values())
"""

def each_energy(every_mask, every_y):
    y_list = every_y.values.tolist()
    energy_trait_sum = 0
    for i in range(len(every_mask)):
        if every_mask[i]==mask[i]:
            energy_trait_sum += 1
    return energy_trait_sum

In [10]:
"""
argument: s is integer
procedure: convert s from 0 to 1 or 1 to 0
return: changed value
"""
def y_renewal(s: int):
    return 1-s

In [11]:
from sklearn.utils.extmath import safe_sparse_dot, row_norms
"""
def compute_corr(y, X):
    X_norms = 
    corr = safe_sparse_dot(y, X)
    corr /= X_norms
    X_norms_inv = np.zeros_like(X_norms)
    mask = (X_norms != 0.)
    X_norms_inv[mask] = 1. / X_norms[mask]
    corr *= X_norms_inv
    corr /= np.linalg.norm(y)#ベクトルの絶対値（原点からの距離）
    # convert to p-value
    degrees_of_freedom = y.size - (2 if center else 1)
    return corr
"""

'\ndef compute_corr(y, X):\n    X_norms = \n    corr = safe_sparse_dot(y, X)\n    corr /= X_norms\n    X_norms_inv = np.zeros_like(X_norms)\n    mask = (X_norms != 0.)\n    X_norms_inv[mask] = 1. / X_norms[mask]\n    corr *= X_norms_inv\n    corr /= np.linalg.norm(y)#ベクトルの絶対値（原点からの距離）\n    # convert to p-value\n    degrees_of_freedom = y.size - (2 if center else 1)\n    return corr\n'

In [12]:
def Montecarlo(T, num_select, MCMC_step):    
    E = []
    y_go = []
    y_list = y.values.tolist()
    
    #元のyを使って
    each_energy(mask, ori_y) #= len(mask)= 20
    E.append(0)
    y_go.append(y_list)
    ori_y = y
    
    for j in range(MCMC_step):    
        i = random.randrange(num_sample)
        #print(i)
        ori_yi = y[i]
        y[i] = y_renewal(y[i])
        if np.all(np.array(y) == 1) or np.all(np.array(y) == 0):
            #print("warning occured")
            continue
        with warnings.catch_warnings():
            warnings.filterwarnings('error')
            try:
                mask_new = selection(num_select, X, y)
            except Warning:
                print("warning is here.")
                print(y)
                continue
        E_now = each_energy(mask, ori_y) - each_energy(mask_new, y)
        E_last = E[-1]
        r = math.exp(-(E_now-E_last)/T[j]) #温度付き
        R = np.random.uniform(0.0, 1.0)
        if R > r :
            y[i] = ori_yi #元に戻す       
        else:
            E.append(E_now)
            y_list = y.values.tolist()
            y_go.append(y_list)
    return E, y_go #エネルギーのリスト、ｙのリスト

In [13]:
E, y_go = Montecarlo(T,num_select,MCMC_step)
#print(E)
#print(len(E)==len(y_go))
#print(y_go)
result_df = pd.DataFrame(y_go)
result_df['Energy'] = E
result_df.to_csv("simulated_annealing_trial1210.csv", sep=",") #yが一要素も変わっていない
print(result_df)

E_Xaxis = [i for i in range(len(E))]
plt.plot(E_Xaxis, E)
plt.show()

UnboundLocalError: local variable 'ori_y' referenced before assignment

In [ ]:
mask_new = selection(num_select, X, y)

In [ ]:
y

In [ ]:
from sklearn.utils.extmath import safe_sparse_dot, row_norms
from scipy.sparse import issparse

In [ ]:
issparse(X)

In [ ]:
y_ = y - np.mean(y)
X_means = X.mean(axis=0)
X_norms = np.sqrt(row_norms(X.T, squared=True) -
                    X.shape[0] * X_means ** 2)

In [ ]:
X_norms

In [ ]:
corr = safe_sparse_dot(y_, X)
print(corr)
corr /= X_norms
print(corr)
corr /= np.linalg.norm(y_)
print(corr)
print("hoge")
print(corr**2)

In [ ]:
degrees_of_freedom = y.size - (2 if True else 1)
F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
print(F)

In [ ]:
safe_sparse_dot(y, X)

###  results
* yに変化なし…MCMC_step=1000, 5000, 10000のとき

* 　`else:`
       `E.append(E_now)`
       `y_list = y.values.tolist()`
       `y_go.append(y_list)`
       
  と改善したら良くなった


* マイナスのエネルギーはおかしい
  * each_energy(mask_new, y)に問題があるだろう